In [21]:
import os
import pandas as pd
import pdb

In [71]:
win_rates = pd.read_csv('win_rates.csv')
scraped_games = pd.read_csv('scraped_games.csv')

actual_win = []; predicted_win = []
win_rate_1_list = []; win_rate_2_list = []
win_diff_list = []
for idx, row in scraped_games.iterrows():
    score_1 = row['Score 1']
    score_2 = row['Score 2']
    
    team_1 = row['Team 1']
    team_2 = row['Team 2']
    
    # add winner
    if score_1 > score_2:
        winner = team_1
    else:
        winner = team_2
    
    actual_win.append(winner)
    
    # add predicted winner
    win_rate_1 = win_rates[win_rates['NFL Name']==team_1]['Win Rate'].iloc[0]
    win_rate_2 = win_rates[win_rates['NFL Name']==team_2]['Win Rate'].iloc[0]
    
    if win_rate_1 > win_rate_2:
        winner = team_1
    else:
        winner = team_2
    
    predicted_win.append(winner)
    win_rate_1_list.append(win_rate_1.round(3))
    win_rate_2_list.append(win_rate_2.round(3))

    win_diff = abs(win_rate_1.round(3) - win_rate_2.round(3))
    win_diff_list.append(win_diff)
    
actual_df = pd.DataFrame(actual_win, columns=['Actual'])
predicted_df = pd.DataFrame(predicted_win, columns=['Predicted'])
win_rate_1_df = pd.DataFrame(win_rate_1_list, columns=['Win Rate 1'])
win_rate_2_df = pd.DataFrame(win_rate_2_list, columns=['Win Rate 2'])
win_diff_df = pd.DataFrame(win_diff_list, columns=['Win Diff'])

correct_df = pd.DataFrame(actual_df['Actual'] == predicted_df['Predicted'],columns=['Pred Correct'])

scraped_games = pd.concat([scraped_games, win_rate_1_df, win_rate_2_df, predicted_df, correct_df, win_diff_df], axis=1)
scraped_games.to_csv('scraped_games_pred.csv', index=None)

In [72]:
weeks = scraped_games['Week'].drop_duplicates().to_list()

percent_list = []
for week in weeks:
    mid_df = scraped_games[scraped_games['Week']==week]
    numerator = mid_df[mid_df['Pred Correct']==True].shape[0]
    denominator = mid_df.shape[0]
    percent = numerator / denominator
    
    lose = denominator - numerator
        
    mid_dict = {'Week':week,
                '# Games':denominator,
                'Win':numerator,
                'Lose':lose,
                'Percent':percent}
    
    percent_list.append(mid_dict)
    
df_week = pd.DataFrame(percent_list)[['Week','# Games','Win','Lose','Percent']]

In [84]:
numerator = scraped_games[scraped_games['Pred Correct']==True].shape[0]
denominator = scraped_games.shape[0]
percent = numerator / denominator

print(percent)
scraped_games['Pred Correct'].value_counts()

0.5512820512820513


True     86
False    70
Name: Pred Correct, dtype: int64

In [85]:
df_week.to_clipboard()

In [82]:
print(scraped_games[scraped_games['Pred Correct']==False]['Win Diff'].mean())
print(scraped_games[scraped_games['Pred Correct']==True]['Win Diff'].mean())

0.060328571428571426
0.05848837209302324
